In [1]:
import os, sys
sys.path.insert(0, '/home/trax/trax/website/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "trax.settings")
import django
django.setup()

In [2]:
from events.models import HotlappingLaptime, Hotlapping
from vehicles.models import Vehicle
from tracks.models import Track, Laptime
from players.models import Player
import datetime
import xml.etree.ElementTree as ET
tree = ET.parse('/home/trax/tepcottbeautified.html')
root = tree.getroot()
track = Track.objects.get(title__startswith="EpsiloShark")
event = Hotlapping.objects.get(pk=16)

In [12]:
u = Player.objects.get(username__startswith="justus")
print([(x.pk, x.laptime.duration) for x in HotlappingLaptime.objects.filter(laptime__player=u, laptime__track=track)])

[(403, '1:44.129'), (404, '1:44.129'), (405, '1:44.129'), (406, '1:44.129'), (407, '1:44.129'), (408, '1:44.129')]


In [11]:
for x in HotlappingLaptime.objects.filter(laptime__player=u, laptime__track=track):
    print (x.laptime.millis)
    x.laptime.millis += 1
    x.laptime.save()

104128
104128
104128
104128
104128
104128


In [13]:
def doit(o):
    """parses the xml objects and creates a Laptime object"""
    try:
        user = Player.objects.get(username=tr.getchildren()[1].text)
    except:
        user = Player.objects.get(username="abe.cede")
    vehicle = Vehicle.objects.get(name=tr.getchildren()[2].text)
    link = tr.getchildren()[3].getchildren()[0].attrib['href']
    created = datetime.date(*[int(x) for x in tr.getchildren()[4].text.strip().split('-')])
    mins, secs = tr.getchildren()[5].text.strip().split(':')
    laptime_secs = int(mins) * 60 + float(secs)
    lt = Laptime(track=track, player=user, created=created, recorded=created,
                 vehicle=vehicle, millis=laptime_secs*1000, link=link,
                millis_per_km = laptime_secs*1000/track.route_length_km)
    lt.save()
    return lt

In [14]:
for tr in root.iter("tr"):
    for td in tr:
        lt = doit(tr)
        hll = HotlappingLaptime(approved=True, hotlapping=event, laptime=lt)
        hll.save()

/home/trax/trax/website/players/utils.py:65: RuntimeWarning: divide by zero encountered in true_divide
  modified_z_score = 0.6745 * diff / med_abs_deviation
/home/trax/trax/website/players/utils.py:65: RuntimeWarning: invalid value encountered in true_divide
  modified_z_score = 0.6745 * diff / med_abs_deviation
/home/trax/trax/website/players/utils.py:66: RuntimeWarning: invalid value encountered in greater
  return modified_z_score > thresh


In [15]:
xx = Hotlapping.objects.get(pk=17)
xx.delete()

(43,
 {'events.Hotlapping': 1,
  'events.HotlappingLaptime': 0,
  'events.Hotlapping_vehicles': 42})